# 07 Marginal and conditional distributions

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.0.1`
import $ivy.`org.apache.spark::spark-mllib:3.0.1`
import $ivy.`org.plotly-scala::plotly-almond:0.8.0`

import $ivy.$                                  

import $ivy.$                                    

import $ivy.$                                      

In [14]:
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
import org.apache.spark.sql._
import org.apache.spark.sql.{functions => F}
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature.StringIndexer
import scala.collection.mutable.LinkedHashMap
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
repl.pprinter() = repl.pprinter().copy(defaultHeight = 1)

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import org.apache.spark.sql._

import org.apache.spark.sql.{functions => F}

import org.apache.spark.sql.types._

import org.apache.spark.ml.feature.StringIndexer

import scala.collection.mutable.LinkedHashMap

import org.apache.log4j.{Level, Logger}


[khanacademy](https://www.khanacademy.org/math/ap-statistics/analyzing-categorical-ap/distributions-two-way-tables/v/marginal-distribution-and-conditional-distribution?modal=1)

![Marginal and conditional distributions fig 1](./imgs/01-07-01.png)

In [3]:
val spark = NotebookSparkSession.builder.appName("statistics").master("local").getOrCreate()

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@7b023175

In [4]:
import spark.implicits._

import spark.implicits._

In [6]:
val dataset = LinkedHashMap(
    "#" -> Seq("80-100", "60-79", "40-59", "20-39", "0-19"),
    "0-20" -> Seq(0, 0, 2, 10, 2),
    "21-40" -> Seq(4, 20, 4, 2, 0),
    "41-60" -> Seq(16, 30, 32, 8, 0),
    ">60" -> Seq(20, 10, 32, 0, 8),
)

dataset: LinkedHashMap[String, Seq[Any]] = ...

In [15]:
var sdf = dataset.values.toList.transpose.map({case List(a: String, b: Int, c: Int, d: Int, e: Int) => (a, b, c, d, e)}).toDF(dataset.keys.toList:_*)
sdf.registerTempTable("sdf_table")
sdf.show()

+------+----+-----+-----+---+
|     #|0-20|21-40|41-60|>60|
+------+----+-----+-----+---+
|80-100|   0|    4|   16| 20|
| 60-79|   0|   20|   30| 10|
| 40-59|   2|    4|   32| 32|
| 20-39|  10|    2|    8|  0|
|  0-19|   2|    0|    0|  8|
+------+----+-----+-----+---+



sdf: DataFrame = [#: string, 0-20: int ... 5 more fields]

In [28]:
sdf = sdf.withColumn("Total", F.col("0-20") + F.col("21-40") + F.col("41-60") + F.col(">60"))
sdf = sdf.withColumn("Percentage", F.col("Total") / sdf.select(F.sum("Total")).collect()(0)(0).asInstanceOf[Long] * 100)
sdf.show()

collect at cmd27.sc:2

1 / 1

collect at cmd27.sc:2

1 / 1

+------+----+-----+-----+---+-----+----------+
|     #|0-20|21-40|41-60|>60|Total|Percentage|
+------+----+-----+-----+---+-----+----------+
|80-100|   0|    4|   16| 20|   40|      20.0|
| 60-79|   0|   20|   30| 10|   60|      30.0|
| 40-59|   2|    4|   32| 32|   70|      35.0|
| 20-39|  10|    2|    8|  0|   20|      10.0|
|  0-19|   2|    0|    0|  8|   10|       5.0|
+------+----+-----+-----+---+-----+----------+



In [72]:
sdf = sdf.withColumn("Total", F.col("0-20") + F.col("21-40") + F.col("41-60") + F.col(">60"))
sdf = sdf.withColumn("Percentage", F.col("Total")/sdf.select(F.sum("Total")).collect()[0][0] * 100)
sdf.show()

+------+----+-----+-----+---+-----+----------+
|     #|0-20|21-40|41-60|>60|Total|Percentage|
+------+----+-----+-----+---+-----+----------+
|80-100|   0|    4|   16| 20|   40|      20.0|
| 60-79|   0|   20|   30| 10|   60|      30.0|
| 40-59|   2|    4|   32| 32|   70|      35.0|
| 20-39|  10|    2|    8|  0|   20|      10.0|
|  0-19|   2|    0|    0|  8|   10|       5.0|
+------+----+-----+-----+---+-----+----------+



In [29]:
sdf.select("#", "Total", "Percentage").show()

+------+-----+----------+
|     #|Total|Percentage|
+------+-----+----------+
|80-100|   40|      20.0|
| 60-79|   60|      30.0|
| 40-59|   70|      35.0|
| 20-39|   20|      10.0|
|  0-19|   10|       5.0|
+------+-----+----------+



In [31]:
sdf.select("#", "41-60")
    .withColumn("Percentage", F.col("41-60") / sdf.select(F.sum("41-60")).collect()(0)(0) * 100).show()

collect at cmd30.sc:2

1 / 1

collect at cmd30.sc:2

1 / 1

+------+-----+------------------+
|     #|41-60|        Percentage|
+------+-----+------------------+
|80-100|   16|  18.6046511627907|
| 60-79|   30|34.883720930232556|
| 40-59|   32|  37.2093023255814|
| 20-39|    8|  9.30232558139535|
|  0-19|    0|               0.0|
+------+-----+------------------+

